In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import gensim
from gensim import corpora
from pprint import pprint

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [3]:
s = open("C:/Users/1/Desktop/de.txt", "r", encoding="utf-8")
s = s.read()
if '\xad' in s:
    s = s.replace("\xad", "")
documents = s.split("\n\n \n\n")

In [4]:
print('Text collection size and median length in symbols:')
print(len(documents), np.median([len(d) for d in documents]))

Text collection size and median length in symbols:
1439 8027.0


In [5]:
import string
from nltk.corpus import stopwords
translator = str.maketrans('', '', string.punctuation)
stop_words = stopwords.words('german')

In [6]:
def preprocess_doc(word, translator, stop_words):
    word = word.translate(translator)
    word = word.lower()
    if word not in stop_words:
        return(word)
        
documents_preprocessed = []
for doc in documents:
    doc = doc.split()
    doc_new = []
    for word_doc in doc:
        if preprocess_doc(word_doc, translator, stop_words) != None:
            doc_new.append(preprocess_doc(word_doc, translator, stop_words))
    doc_new = " ".join(doc_new)
    documents_preprocessed.append(doc_new)

In [7]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                   min_df=5)
tfidf = tfidf_vectorizer.fit_transform(documents_preprocessed)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [8]:
n_topics = 10
nmf = NMF(n_topics)
nmf_doc_topic = nmf.fit_transform(tfidf)
print('NMF doc-topic shape:', nmf_doc_topic.shape)

NMF doc-topic shape: (1439, 10)


In [9]:
tf_vectorizer = CountVectorizer(max_df=0.8,
                                min_df=5)
tf = tf_vectorizer.fit_transform(documents_preprocessed)
tf_feature_names = tf_vectorizer.get_feature_names()

In [10]:
lda = LatentDirichletAllocation(n_topics)
lda_doc_topic = lda.fit_transform(tf)
print('LDA doc-topic shape:', lda_doc_topic.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LDA doc-topic shape: (1439, 10)


In [11]:
no_top_words = 10
print('\nNMF top terms:')
display_topics(nmf, tfidf_feature_names, no_top_words)
print('\nLDA top terms:')
display_topics(lda, tf_feature_names, no_top_words)


NMF top terms:
Topic 0:
unternehmen, müssen, gesellschaft, brauchen, wirtschaft, gibt, politik, arbeit, muß, bürger
Topic 1:
europa, europäischen, europäische, union, europas, verfassung, mitgliedstaaten, staaten, europäer, bürger
Topic 2:
juden, geschichte, leben, israel, jüdische, christen, erinnerung, kirche, jüdischen, opfer
Topic 3:
afrika, afrikanischen, afrikas, afrikanische, partnerschaft, kontinent, armut, afrikaner, entwicklung, müssen
Topic 4:
wissenschaft, forschung, hochschulen, wissenschaftlichen, wissenschaftler, universität, universitäten, wissenschaftliche, wissenschaftsrat, wissenschaften
Topic 5:
kinder, schulen, schule, bildung, lehrer, schüler, eltern, lernen, müssen, brauchen
Topic 6:
japan, beziehungen, deutsche, china, präsident, land, zusammenarbeit, mexiko, indien, länder
Topic 7:
kunst, musik, kultur, ganz, frau, abend, herr, theater, lieber, schloss
Topic 8:
polen, polnischen, deutschpolnischen, deutschpolnische, deutschen, deutsche, polnische, nachbarn, po

In [12]:
import re
tok_collection = []
for d in documents_preprocessed:
    tok_collection.append([w for w in re.split('[\W]+', d) if len(w) > 4])

In [13]:
dictionary = corpora.Dictionary(tok_collection)
corpus = [dictionary.doc2bow(text) for text in tok_collection]

In [14]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           num_topics=n_topics,
                                           id2word=dictionary)
pprint(ldamodel.print_topics())

[(0,
  '0.005*"menschen" + 0.004*"müssen" + 0.004*"heute" + 0.003*"immer" + '
  '0.003*"deutschland" + 0.003*"schon" + 0.003*"jahren" + 0.003*"zukunft" + '
  '0.002*"unserer" + 0.002*"viele"'),
 (1,
  '0.006*"menschen" + 0.006*"deutschland" + 0.006*"immer" + 0.005*"heute" + '
  '0.005*"viele" + 0.005*"müssen" + 0.004*"schon" + 0.003*"deutschen" + '
  '0.003*"unserer" + 0.003*"deutsche"'),
 (2,
  '0.009*"menschen" + 0.007*"heute" + 0.006*"deutschland" + 0.004*"immer" + '
  '0.004*"deutschen" + 0.004*"dafür" + 0.003*"müssen" + 0.003*"gesellschaft" + '
  '0.003*"viele" + 0.003*"gerade"'),
 (3,
  '0.006*"menschen" + 0.006*"heute" + 0.006*"deutschland" + 0.005*"immer" + '
  '0.004*"viele" + 0.004*"deutschen" + 0.003*"schon" + 0.003*"müssen" + '
  '0.003*"leben" + 0.003*"jahren"'),
 (4,
  '0.008*"menschen" + 0.008*"deutschland" + 0.006*"müssen" + 0.005*"heute" + '
  '0.005*"immer" + 0.004*"deutschen" + 0.004*"viele" + 0.003*"dafür" + '
  '0.003*"europa" + 0.003*"zukunft"'),
 (5,
  '0.006*"he

NMF top terms: наиболее релевантные результаты.
Доделать: лемматизация (!)